# Домашнее задание   1

Имеются данные adult.csv (см. в материалах к занятию).

Целевой переменной является уровень дохода income (крайний правый столбец).

Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html

Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.

In [49]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [21]:
data = pd.read_csv( 'adult.csv' )
data.head(50)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [51]:
# Вибирем признаки, по которым будем стросить прогноз дохода:
selectedColumns = data[ [ 'age', 'workclass', 'educational-num', 'marital-status', 'occupation', 'relationship', 'hours-per-week', 'native-country', 'income'] ]
selectedColumns.head()

,age,workclass,educational-num,marital-status,occupation,relationship,hours-per-week,native-country,income
0,25,Private,7,Never-married,Machine-op-inspct,Own-child,40,United-States,<=50K
1,38,Private,9,Married-civ-spouse,Farming-fishing,Husband,50,United-States,<=50K
2,28,Local-gov,12,Married-civ-spouse,Protective-serv,Husband,40,United-States,>50K
3,44,Private,10,Married-civ-spouse,Machine-op-inspct,Husband,40,United-States,>50K
4,18,?,10,Never-married,?,Own-child,30,United-States,<=50K


In [52]:
data_1 = selectedColumns 

In [53]:
data_1.replace('?', np.nan, inplace=True)
data_1.head(50)

C:\Users\Reb\anaconda3\lib\site-packages\pandas\core\frame.py:4383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,age,workclass,educational-num,marital-status,occupation,relationship,hours-per-week,native-country,income
0,25,Private,7,Never-married,Machine-op-inspct,Own-child,40,United-States,<=50K
1,38,Private,9,Married-civ-spouse,Farming-fishing,Husband,50,United-States,<=50K
2,28,Local-gov,12,Married-civ-spouse,Protective-serv,Husband,40,United-States,>50K
3,44,Private,10,Married-civ-spouse,Machine-op-inspct,Husband,40,United-States,>50K
4,18,NaN,10,Never-married,NaN,Own-child,30,United-States,<=50K
5,34,Private,6,Never-married,Other-service,Not-in-family,30,United-States,<=50K
6,29,NaN,9,Never-married,NaN,Unmarried,40,United-States,<=50K
7,63,Self-emp-not-inc,15,Married-civ-spouse,Prof-specialty,Husband,32,United-States,>50K
8,24,Private,10,Never-married,Other-service,Unmarried,40,United-States,<=50K
9,55,Private,4,Married-civ-spouse,Craft-repair,Husband,10,United-States,<=50K


In [55]:
data_1 = data_1.rename(columns={'educational-num': 'educational_num', 'marital-status': 'marital_status', 'hours-per-week': 'hours_per_week', 'native-country': 'native_country'})

In [89]:
data_1.isnull().sum()

age                   0
workclass          2799
educational_num       0
marital_status        0
occupation         2809
relationship          0
hours_per_week        0
native_country      857
income                0
dtype: int64

In [88]:
data_1.shape

(48842, 9)

In [90]:
#Удалим строки, в которых есть хотя бы одно пропущенное значение

data_2 = data_1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data_2.head(50)

,age,workclass,educational_num,marital_status,occupation,relationship,hours_per_week,native_country,income
0,25,Private,7,Never-married,Machine-op-inspct,Own-child,40,United-States,<=50K
1,38,Private,9,Married-civ-spouse,Farming-fishing,Husband,50,United-States,<=50K
2,28,Local-gov,12,Married-civ-spouse,Protective-serv,Husband,40,United-States,>50K
3,44,Private,10,Married-civ-spouse,Machine-op-inspct,Husband,40,United-States,>50K
5,34,Private,6,Never-married,Other-service,Not-in-family,30,United-States,<=50K
7,63,Self-emp-not-inc,15,Married-civ-spouse,Prof-specialty,Husband,32,United-States,>50K
8,24,Private,10,Never-married,Other-service,Unmarried,40,United-States,<=50K
9,55,Private,4,Married-civ-spouse,Craft-repair,Husband,10,United-States,<=50K
10,65,Private,9,Married-civ-spouse,Machine-op-inspct,Husband,40,United-States,>50K
11,36,Federal-gov,13,Married-civ-spouse,Adm-clerical,Husband,40,United-States,<=50K


In [62]:
target = data_2['income']
data_ml = data_2[[ 'age', 'workclass', 'educational_num', 'marital_status', 'occupation', 'relationship', 'hours_per_week', 'native_country' ]]

In [91]:
# столбцы workclass, education_num, marital_status, oc'cupation, relationship, native_country являются категориальными переменными# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( data_2, columns = [ 'workclass', 'marital_status', 'occupation', 'relationship', 'native_country' ] )
X

,age,educational_num,hours_per_week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,25,7,40,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,9,50,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,12,40,>50K,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,10,40,>50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,6,30,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,12,38,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48838,40,9,40,>50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48839,58,9,40,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48840,22,9,20,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [63]:
le.fit( target )

LabelEncoder()

In [67]:
y = pd.Series( data = le.transform( target ))
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [73]:
lr = LogisticRegression()

In [74]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 145)

In [75]:
lr.fit(X_train, y_train)

C:\Users\Reb\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [76]:
lr.score(X_test, y_test)

0.8296296296296296

In [77]:
lr.score(X_train, y_train)

0.8306382508223457

## SVM

In [82]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


In [84]:
clf = make_pipeline(StandardScaler(), SVC(gamma = 'auto') )
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [85]:
clf.score(X_train, y_train)

0.8315227907233878

In [86]:
clf.score(X_test, y_test)

0.8247650635710337